In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df
import xarray as xr
import netCDF4 as nc
import os
import requests
from datetime import timedelta
import tensorflow as tf
from datetime import datetime

In [ ]:
ds_SALT = xr.open_dataset("../SODA/salt_combined.nc" , decode_times=False)

In [ ]:
#### 

In [ ]:
df_VK08_TCCount = pd.read_table('../SODA/VK08_TCCounts_1878-2020.dat', sep="\s+", header=None)
df_VK08_TCCount = df_VK08_TCCount.rename(columns={0: "Year", 1:"VK08_TCCount"}).copy()
ds_SST = xr.open_dataset("../SODA/SST.nc" , decode_times=True)

ds_ssh = xr.open_dataset("../SODA/data.nc" , decode_times=False)
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_ssh = fix_calendar(ds_ssh)
ds_ssh = xr.decode_cf(ds_ssh)


In [ ]:
ds_SST_sub = ds_SST.where(((ds_SST.lon>40 )&(ds_SST.lon<80) & (ds_SST.lat>10 )&(ds_SST.lat<20)  ), drop=True)
ds_ssh = ds_ssh.where(((ds_ssh.lon > 40 ) & (ds_ssh.lon < 80 ) & (ds_ssh.lat > 10) & (ds_ssh.lat <20)), drop=True)

df_sst = pd.DataFrame()
df_sst['time'] = np.array((ds_SST_sub['time']))
df_sst['sst'] = ''
for i in range(len(df_sst)):
  #df_sst['time'][i] = pd.Timestamp(1891, 1, 1) + timedelta(days= df_sst['time'][i])
  df_sst['sst'][i] = np.array((ds_SST_sub['sst'][i]))
    
df_ssh= pd.DataFrame()
np.array((ds_ssh['time'][0]))
df_ssh['time'] = np.array((ds_ssh['time']))
df_ssh['ssh'] = ''
for i in range(1656):
  df_ssh['ssh'][i] = np.array((ds_ssh['ssh'][i]))


In [ ]:
# Interpolation
from scipy.interpolate import NearestNDInterpolator
for i in range(len(df_sst)):
  data = df_sst['sst'][i] 
  mask = np.where(~np.isnan(data))
  interp = NearestNDInterpolator(np.transpose(mask), data[mask])
  filled_data = interp(*np.indices(data.shape))
  df_sst['sst'][i] = filled_data

for i in range(len(df_ssh)):
  data = df_ssh['ssh'][i] 
  mask = np.where(~np.isnan(data))
  interp = NearestNDInterpolator(np.transpose(mask), data[mask])
  filled_data = interp(*np.indices(data.shape))
  df_ssh['ssh'][i] = filled_data

In [ ]:
def convert_to_dt(x):
    return datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')


df_sst["time"] = df_sst.time.apply(convert_to_dt)
df_sst['Month'] = 0
df_sst['Year'] = 0
for i in range(len(df_sst)):
    df_sst['Month'][i] =df_sst['time'][i].month
    df_sst['Year'][i] =df_sst['time'][i].year
    
df_ssh["time"] = df_ssh.time.apply(convert_to_dt)
df_ssh['Month'] = 0
df_ssh['Year'] = 0
for i in range(len(df_ssh)):
    df_ssh['Month'][i] =df_ssh['time'][i].month
    df_ssh['Year'][i] =df_ssh['time'][i].year

In [ ]:
df_sst

In [ ]:
df_ssh['ssh'][0].shape

In [ ]:
df_VK08_TCCount

In [ ]:
df_combined = pd.merge(df_sst,df_ssh[['ssh','Year','Month']],how = 'inner',on = ['Month','Year']).drop(axis = 1,labels = ['Month'])

In [ ]:
df_combiend

In [ ]:
### DATA EXPLORATION ###

In [ ]:
# Plot of TC Count
df_VK08_TCCount[ '7year_rolling_avg' ] = df_VK08_TCCount.VK08_TCCount.rolling(5).mean()
plt.figure(figsize=(20,10))
plt.plot(df_VK08_TCCount['Year'],df_VK08_TCCount['VK08_TCCount'])
plt.plot(df_VK08_TCCount['Year'],df_VK08_TCCount['7year_rolling_avg'])
plt.gca().set(title='TC Count Visulization Over Time', xlabel="Year", ylabel="TC Count")

In [ ]:
plt.boxplot(df_VK08_TCCount['VK08_TCCount'])

In [ ]:
lon = 60 - 40
lat = 15 - 10
ssh_lon = lon*2
ssh_lat = lat*2

# Plot of SSH and SST in one location
# df_VK08_TCCount[ '7year_rolling_avg' ] = df_VK08_TCCount.VK08_TCCount.rolling(5).mean()
temp_sst = []
temp_ssh = []
for i in range(len(df_combined)):
    temp_sst.append(df_combined['sst'][i][lat][lon])
    temp_ssh.append(df_combined['ssh'][i][ssh_lat][ssh_lon])
w = 24
temp_sstAvg = [0] * (w-1)
temp_sshAvg = [0] * (w-1)
temp_sstAvg.extend(np.convolve(temp_sst, np.ones(w), 'valid') / w)
temp_sshAvg.extend(np.convolve(temp_ssh, np.ones(w), 'valid') / w)
plt.figure(figsize=(20,10))
plt.plot(df_combined['time'],temp_sst / np.linalg.norm(temp_sst) + 0.02,color='r')
plt.plot(df_combined['time'],temp_sstAvg / np.linalg.norm(temp_sst) + 0.02)
plt.plot(df_combined['time'],temp_ssh / np.linalg.norm(temp_ssh),color = 'b')
plt.plot(df_combined['time'],temp_sshAvg / np.linalg.norm(temp_ssh))
df_VK08_TCCount[ '7year_rolling_avg' ] = df_VK08_TCCount.VK08_TCCount.rolling(5).mean()

In [ ]:
from scipy.stats.stats import pearsonr   
print(pearsonr(temp_sstAvg,temp_sshAvg))